Importação das bibliotecas que serão utilizadas para a criação do algoritmo de Sumarização Abstrata

In [1]:
import numpy as np
import pandas as pd
import spacy
import string
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
import tensorflow_datasets as tfds
import random
from sklearn.model_selection import train_test_split
import re

In C:\Users\renat\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\renat\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\renat\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\renat\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\renat\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
def tokenize(lang, max_seq):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True, filters='',
                                                         num_words=2**16)
  lang_tokenizer.fit_on_texts(lang)



  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post',
                                                         maxlen=max_seq)

  return tensor, lang_tokenizer

def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

Carregado o dataset com as notícias, separando o texto e o título

In [4]:
data = pd.read_json('tcc1.json', encoding='utf-8')

noticias = ['<start> ' + m + ' <end>' for m in data.texto.tolist()]
titulos  = ['<start> ' + m + ' <end>' for m in data.título.tolist()]

input_tensor, inp_lang = tokenize(noticias, max_seq=600)
target_tensor, targ_lang = tokenize(titulos, max_seq=20)


In [22]:
titulos

['<start> Falta de política clara para conter pandemia atrasa retomada da economia, dizem empresários  <end>',
 '<start> Brasil chega a 16.792 mortes e se torna 3º do mundo com mais casos <end>',
 '<start> MPF investigará se Flávio Bolsonaro foi avisado de operação da PF <end>',
 "<start> Advogado investigado por 'rachadinha' é suspeito de presenciar suposto vazamento <end>",
 '<start> Celso de Mello diz que decidirá sobre sigilo do vídeo até o fim da semana <end>',
 "<start> Empresa anuncia resultados 'positivos preliminares' em teste de vacina <end>",
 "<start> Vereadores aprovam 'feriadão' em SP para tentar aumentar isolamento <end>",
 '<start> Hospital de referência em Natal fecha as portas do PS <end>',
 '<start> Nem novos leitos dão conta do avanço do coronavírus no Ceará <end>',
 '<start> Maranhão planeja reabertura na próxima semana, diz governador <end>',
 "<start> Enfermeiros e médicos do RJ mostram rotina: '6 horas sem beber água' <end>",
 '<start> Governador de Pernambuco t

In [5]:
inp_lang.word_index['<start>']

42

In [6]:
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [7]:
from sklearn.model_selection import train_test_split

# faz separacao 80/20
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# mostra tamanhos
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

229 229 58 58


In [8]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
42 ----> <start>
26 ----> são
20 ----> mais
1 ----> de
571 ----> 16
81 ----> mil
115 ----> mortes
11 ----> no
215 ----> brasil,
32 ----> mas
9 ----> para
15 ----> uma
11959 ----> minoria
705 ----> parece
5 ----> que
317 ----> tanto
11960 ----> faz.
3 ----> o
9123 ----> fantástico
11961 ----> pergunta:
18 ----> por
5 ----> que
16 ----> é
5 ----> que
38 ----> tem
82 ----> gente
5 ----> que
9124 ----> age
23 ----> como
25 ----> se
2 ----> a
60 ----> pandemia
17 ----> não
3396 ----> estivesse
11962 ----> acontecendo?
11 ----> no
61 ----> dia
7 ----> em
5 ----> que
3 ----> o
103 ----> brasil
1203 ----> contou
223 ----> 10
81 ----> mil
525 ----> mortos
18 ----> por
173 ----> coronavírus,
13 ----> um
1645 ----> cliente
593 ----> pediu
11963 ----> vinho
11964 ----> espumante
7 ----> em
13 ----> um
9125 ----> restaurante
1 ----> de
11965 ----> gramado,
91 ----> rio
118 ----> grande
6 ----> do
3072 ----> sul.
19 ----> as
11966 ----> garrafas
54 ----> foram
1

In [9]:
# criando o dataset

BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 16
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 200
units = 1024

vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [10]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([16, 600]), TensorShape([16, 20]))

In [11]:
# bloco do encoder

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [12]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (16, 600, 1024)
Encoder Hidden state shape: (batch size, units) (16, 1024)


In [13]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [14]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (16, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (16, 600, 1)


In [23]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)
    print(context_vector)
    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)
    print(x)
    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [24]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

tf.Tensor(
[[-0.00314646  0.00860887  0.00222098 ...  0.00997951  0.00246175
  -0.00527798]
 [-0.00207971  0.00589338  0.00118219 ...  0.00599809  0.00191839
  -0.00283943]
 [-0.00297884  0.00742073  0.00155249 ...  0.00870708  0.00184641
  -0.00463275]
 ...
 [-0.00241177  0.00423362 -0.00014944 ...  0.00373702  0.00081356
  -0.00189337]
 [-0.00204971  0.00039885 -0.00046536 ... -0.00048837 -0.00015328
  -0.00069056]
 [-0.00143831  0.00043566 -0.00035437 ... -0.00093647  0.00026161
  -0.00113795]], shape=(16, 1024), dtype=float32)
tf.Tensor(
[[[-0.01127663 -0.0161803   0.00200553 ...  0.03489823 -0.04394979
   -0.03381564]]

 [[-0.01127663 -0.0161803   0.00200553 ...  0.03489823 -0.04394979
   -0.03381564]]

 [[-0.01127663 -0.0161803   0.00200553 ...  0.03489823 -0.04394979
   -0.03381564]]

 ...

 [[-0.01127663 -0.0161803   0.00200553 ...  0.03489823 -0.04394979
   -0.03381564]]

 [[-0.01127663 -0.0161803   0.00200553 ...  0.03489823 -0.04394979
   -0.03381564]]

 [[-0.01127663 -0.016

In [25]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [26]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [27]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [70]:
print(targ_lang.word_index['<start>'])
print([targ_lang.word_index['<start>']] * BATCH_SIZE)
print(targ_lang.word_index['<start>'] * BATCH_SIZE)

1
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
16


In [29]:
import time

EPOCHS = 200

loss_acumulado = []

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  loss_acumulado.append(total_loss / steps_per_epoch)
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

checkpoint.save(file_prefix = checkpoint_prefix)

Tensor("decoder_1/bahdanau_attention_2/Sum:0", shape=(16, 1024), dtype=float32)
Tensor("decoder_1/embedding_2/embedding_lookup/Identity_1:0", shape=(16, 1, 200), dtype=float32)
Tensor("decoder_1_1/bahdanau_attention_2/Sum:0", shape=(16, 1024), dtype=float32)
Tensor("decoder_1_1/embedding_2/embedding_lookup/Identity_1:0", shape=(16, 1, 200), dtype=float32)
Tensor("decoder_1_2/bahdanau_attention_2/Sum:0", shape=(16, 1024), dtype=float32)
Tensor("decoder_1_2/embedding_2/embedding_lookup/Identity_1:0", shape=(16, 1, 200), dtype=float32)
Tensor("decoder_1_3/bahdanau_attention_2/Sum:0", shape=(16, 1024), dtype=float32)
Tensor("decoder_1_3/embedding_2/embedding_lookup/Identity_1:0", shape=(16, 1, 200), dtype=float32)
Tensor("decoder_1_4/bahdanau_attention_2/Sum:0", shape=(16, 1024), dtype=float32)
Tensor("decoder_1_4/embedding_2/embedding_lookup/Identity_1:0", shape=(16, 1, 200), dtype=float32)
Tensor("decoder_1_5/bahdanau_attention_2/Sum:0", shape=(16, 1024), dtype=float32)
Tensor("decoder_1

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[16,600,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node Mul_183 (defined at <ipython-input-27-49d4266c745b>:26) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[concat_19/_212]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[16,600,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node Mul_183 (defined at <ipython-input-27-49d4266c745b>:26) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 succ

In [ ]:
plt.plot(loss_acumulado)

In [ ]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = sentence.lower()

  inputs = inp_lang.texts_to_sequences([sentence])

  #inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]

  print(inputs)

  inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                         maxlen=max_length_inp,
                                                         padding='post')
  print(inputs.shape)
  inputs = tf.convert_to_tensor(inputs)
  print(inputs.shape)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()


def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

mensagem = '<start> ' + data.texto.tolist()[15] + ' <end>'
print(mensagem)

translate(mensagem)
